In [45]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import missingno as msno
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer


# Optional: Configure matplotlib for inline display in Jupyter
%matplotlib inline

#Loading the Data
df = pd.read_csv('brfss_2022_final.csv')

# Display the first few rows of the DataFrame
print(df.head(10))

   DRNK3GE5  SLEPTIM1  EMTSUPRT  INCOME3  SEXVAR
0       NaN       8.0       1.0     99.0       2
1       NaN       6.0       1.0      5.0       2
2       NaN       5.0       2.0     10.0       2
3       NaN       7.0       1.0     77.0       2
4      88.0       9.0       1.0      5.0       2
5       NaN       7.0       2.0     99.0       1
6      88.0       7.0       1.0      8.0       2
7       NaN       8.0       1.0      7.0       2
8       NaN       6.0       1.0      7.0       2
9      88.0       7.0       1.0      7.0       2


# Model Selection

In [48]:
# Logistic Regression
# Why: a good baseline model, interpretable and efficient

In [50]:
# Define features and target
X = df.drop(columns=['DRNK3GE5'])  # Features
y = df['DRNK3GE5']  # Target variable

In [52]:
print(df.head(10))

   DRNK3GE5  SLEPTIM1  EMTSUPRT  INCOME3  SEXVAR
0       NaN       8.0       1.0     99.0       2
1       NaN       6.0       1.0      5.0       2
2       NaN       5.0       2.0     10.0       2
3       NaN       7.0       1.0     77.0       2
4      88.0       9.0       1.0      5.0       2
5       NaN       7.0       2.0     99.0       1
6      88.0       7.0       1.0      8.0       2
7       NaN       8.0       1.0      7.0       2
8       NaN       6.0       1.0      7.0       2
9      88.0       7.0       1.0      7.0       2


In [ ]:
# Random Forest Classifier
# Why: Captures the nonlinear relationships and handles feature importance well

In [ ]:
# XGBoost Classifier
# Why: Based on gradient boosting, widely used for classification and regression tasks
# because of its high performance, speed 
# works by building multiple weak models sequentially and each new tree tries to correct the errors of the previous one
# it then adjusts weights in order to minimize errors
# prevents overfitting with L1 (lasso) and L2 (ridge) regularization
# also automatically handles missing data 
# lastly, it handles large data efficiently and this dataset is enormous